# Adaptive Sampling

In this tutorial you will learn:
    
    ...
    
    

In [ ]:
from bluesky import RunEngine
from bluesky.plans import count
from simulated_hardware import detector, load_sample, unload_sample, light
from bluesky_widgets.models.plot_builders import Images
from bluesky_widgets.jupyter.figures import JupyterFigure
from bluesky_widgets.utils.streaming import stream_documents_into_runs

In [ ]:
model = Images("detector_image")
figure = JupyterFigure(model.figure)
figure

In [ ]:
RE = RunEngine()

In [ ]:
RE(load_sample(1))

In [ ]:
# Turn on light
RE(light(True))

In [ ]:
RE(count([detector]), stream_documents_into_runs(model.add_run))

In [ ]:
image = model.figure.axes[0].artists[0]
image.style.update(clim=(0, 1000))